In [15]:
import re
import pandas as pd
import os
dataset_dir = r"A:\AML project\dataset\IEMOCAP\nas\database\IEMOCAP_full_release"
dataset_annotation_path = r"dialog\EmoEvaluation"
dataset_transcription_path = r"dialog\transcriptions"
sessions = ['Session1','Session2','Session3','Session4','Session5']

In [16]:
for ses in sessions:
    annotation_path = dataset_dir+'\\'+ses+'\\'+dataset_annotation_path
    transcription_path = dataset_dir+'\\'+ses+'\\'+dataset_transcription_path
    annotation_files = [file for file in os.listdir(annotation_path) if file.endswith(".txt")]
    save_path = r"A:\AML project\Emotion-Classification\Text modality\csv files\\"+ses
    try:
        os.mkdir(save_path)
    except:
        os.rmdir(save_path)
        os.mkdir(save_path)
    '''geting the content of the files as [sentences and labels] and saving them to csv file per every diglouge'''
    useful_regex = re.compile(r'\[.+\]\n', re.IGNORECASE)
    for file in annotation_files:
        annotation_dict = {}
        training_data = []
        path = os.path.join(annotation_path, file)
        f = open(path,'r')
        content = f.read()
        data = re.findall(useful_regex, content)
        for line in data[1:]:
            d = line.split()
            annotation_dict[d[3]] = d[4]
        f.close()
        #getting the utterance from transcription folder
        path = os.path.join(transcription_path,file)
        f = open(path,'r')
        content = f.readlines()
        for line in content:
            line = line.split(":")
            U_id = line[0].split()
            try:
                training_data.append([U_id[0],line[1].strip('\n'),annotation_dict[U_id[0]]])
            except:
                training_data.append([U_id[0],line[1].strip('\n'),""])
        #create df for training data and save it as csv
        df = pd.DataFrame(training_data,columns=["Uid","Utterances","Labels"])
        file = file.split('.')
        df.to_csv(save_path+"\\"+file[0]+".csv",index=False)